In [1]:
import numpy as np
import networkx as nx
from itertools import permutations
import time
from GroupingAlgorithm import grouping, TPBgrouping

# Test Grouping algorithm

Notebook to run different tests on the grouping functions. These functions are included in the document __GroupingAlgorithm.py__.

## Test 1: Easy TPB grouping

We start off with a very naive example. We manually create four chains that we know in advance how they should be grouped. This test is performed with the TPBgrouping function, which implements the LDFC algorithm (the same qiskit uses).

In [2]:
start=time.time()
PS=np.array([[1,1,0],[2,2,3],[3,3,3],[0,1,1]])
Colors, Groups, Measurements = TPBgrouping(PS)
print('Execution time', time.time()-start, 'seconds')
print('Colors', Colors)
print('Groups', Groups)
print('Measurements', Measurements)

Execution time 0.002127201994881034 seconds
Colors {1: 0, 2: 1, 0: 2, 3: 2}
Groups [[1], [2], [0, 3]]
Measurements [[[2, [0]], [2, [1]], [3, [2]]], [[3, [0]], [3, [1]], [3, [2]]], [[1, [0]], [1, [1]], [1, [2]]]]


Test is succesful. Chains 0 ($\{X,X,I\}$) and 3 ($\{I,X,X\}$) get grouped. This is the only option allowing only local measurements.

## Test 2: Hard TPB grouping

Much more demanding test, also with the TPBgrouping function but now with 400 Pauli strings, each of them composed of 6 Pauli operators. Te objective of this test is essentially to check the execution time.

In [3]:
start=time.time()
N=6
np.random.seed(0)
PS=np.random.randint(0,4,[400,N])
Colors, Groups, Measurements=TPBgrouping(PS)
print('Execution time', time.time()-start, 'seconds')
print('Number of groups', len(Groups))
print('Strings of the first group', PS[Groups[0],:])
print('Measurements of the first group', Measurements[0])

Execution time 1.7683753420133144 seconds
Number of groups 139
Strings of the first group [[1 1 3 1 3 2]
 [1 1 3 1 3 0]]
Measurements of the first group [[1, [0]], [1, [1]], [3, [2]], [1, [3]], [3, [4]], [2, [5]]]


## Test 3: Easy Bell grouping (all-to-all connectivity)

We introduce by hand 3 strings of three qubits and allow Bell measurements as well as TPB. The example has been puròsedly chosen such that a Bell measurement can group the qubits 0 and 1 of all three chains. 

Test is succesful.

In [4]:
start=time.time()
PS=np.array([[1,1,3],[2,2,3],[3,3,3]])
WC=list(permutations(list(range(3)),2))
Groups, Measurements=grouping(PS,[4,3,2,1],WC)
print('Execution time', time.time()-start, 'seconds')
print('Groups', Groups, 'This means that there is only one groups, with strings 0, 1 and 2.')
print('Measurements', Measurements, 'This means that (for the group 0) the measurement 4 (Bell) should be performed on the qubits 0 and 1, and the measurement 3 (TPBZ) should be performed on qubit 2')

Execution time 0.0015723860124126077 seconds
Groups [[0, 1, 2]] This means that there is only one groups, with strings 0, 1 and 2.
Measurements [[[4, [0, 1]], [3, [2]]]] This means that (for the group 0) the measurement 4 (Bell) should be performed on the qubits 0 and 1, and the measurement 3 (TPBZ) should be performed on qubit 2


##  Test 4: No grouping (all-to-all connectivity)

Test in which we know by hand that no grouping is possible using only TPB+Bell.

In [5]:
start=time.time()
PS=np.array([[1,2,3],[3,2,2],[1,3,2]])
WC=list(permutations(list(range(3)),2))
Groups, Measurements=grouping(PS,[4,3,2,1],WC)#  Only the measurements 4 (Bell) and TPB are considered for grouping). The Bell measurement is the preferential.
print('Execution time', time.time()-start, 'seconds')
print('Groups', Groups, 'This means that there are 3 groups, one for each string.')
print('Measurements', Measurements)
print('Measurements of group 0:', Measurements[0],  'This means that (for the group 0) the measurement 1 (TPBX) should be performed on the qubit 0, the measurement 2 (TPBY) should be performed on the qubit 1 and the measurement 3 (TPBZ) should be performed on the qubit 2.')

Execution time 0.0017528400057926774 seconds
Groups [[0], [1], [2]] This means that there are 3 groups, one for each string.
Measurements [[[1, [0]], [2, [1]], [3, [2]]], [[3, [0]], [2, [1]], [2, [2]]], [[1, [0]], [3, [1]], [2, [2]]]]
Measurements of group 0: [[1, [0]], [2, [1]], [3, [2]]] This means that (for the group 0) the measurement 1 (TPBX) should be performed on the qubit 0, the measurement 2 (TPBY) should be performed on the qubit 1 and the measurement 3 (TPBZ) should be performed on the qubit 2.


## Test 5: Different conectivities

We check that the grouping depends on the connectivity (which qubits in the are allowed to get entangled with which).

First, with this example of 3 Pauli strings and all-to-all conections we check that grouping into just 1 group is possible.

In [6]:
start=time.time()
PS=np.array([[1,1,1],[2,1,2],[3,1,3]])
WC=list(permutations(list(range(3)),2))
Groups, Measurements=grouping(PS,[4,3,2,1],WC)
print('Execution time', time.time()-start, 'seconds')
print('Groups', Groups)
print('Measurements', Measurements)

Execution time 0.0012155469739809632 seconds
Groups [[0, 1, 2]]
Measurements [[[4, [0, 2]], [1, [1]]]]


If we restrict the conectivity between qubits, for the same example the number of groups increases. It goes from 1 group --> to 3 groups.

In [7]:
start=time.time()
PS=np.array([[1,1,1],[2,1,2],[3,1,3]])
WC=[0,1,2,(0,1),(1,0)]
Groups, Measurements=grouping(PS,[4,3,2,1],WC)
print('Execution time', time.time()-start, 'seconds')
print('Groups', Groups)
print('Measurements', Measurements)

Execution time 0.0016435800353065133 seconds
Groups [[0], [1], [2]]
Measurements [[[1, [0]], [1, [1]], [1, [2]]], [[2, [0]], [1, [1]], [2, [2]]], [[3, [0]], [1, [1]], [3, [2]]]]


## Test 6: Hard grouping test. Measurement preference dependence

Here, we show with an example that the grouping algorithm works with hard instances for PS. In addition, we show that the algorithm depends on the preference of assigned measurements.


In [8]:
start=time.time()
N=6
np.random.seed(0)
WC=list(permutations(list(range(N)),2))
AM=[1,2,3,4,5,6,7,8,9] 
PS=np.random.randint(0,4,[400,N])
Groups, Measurements=grouping(PS,AM,WC)
print('Execution time', time.time()-start, 'seconds')
print('Number of groups', len(Groups))
print('Strings of the first group', PS[Groups[0],:])
print('Measurements of the first group', Measurements[0])

Execution time 1.8879329759511165 seconds
Number of groups 80
Strings of the first group [[1 1 3 1 3 2]
 [1 3 1 1 3 2]
 [2 1 3 1 2 2]
 [1 3 1 3 3 3]
 [2 1 3 0 2 0]]
Measurements of the first group [[6, [1, 2]], [6, [0, 4]], [7, [3, 5]]]


In [10]:
start=time.time()
AM=[9,8,7,6,5,4,3,2,1] 
Groups, Measurements=grouping(PS,AM,WC)
print('Execution time', time.time()-start, 'seconds')
print('Number of groups', len(Groups))
print('Strings of the first group', PS[Groups[0],:])
print('Measurements of the first group', Measurements[0])

Execution time 1.9476499669835903 seconds
Number of groups 82
Strings of the first group [[1 1 3 1 3 2]
 [1 3 1 1 3 2]
 [2 1 3 1 2 2]
 [1 3 1 3 3 3]
 [2 1 3 0 2 0]]
Measurements of the first group [[6, [1, 2]], [6, [0, 4]], [7, [3, 5]]]


import qiskit.tools.jupyter
%qiskit_version_table__For a much more careful discussion on how the order in which the measurements are assigned, as well as the order of the Pauli strings, please visit the notebook: Order_Paulis__

In [1]:
import qiskit.tools.jupyter
%qiskit_version_table